# 5 TP sur la génération de codes de Hamming systématiques
#### _Auteurs: Rafael Dousse_

### But
1. Décrire l’algorithme qui permet de construire la matrice génératrice d’un code de Hamming (n, k). Bien exprimer les relations qui lient n et k et les valeurs possibles de ce couple.
2. Ecrire un programme qui génère arbitrairement les matrices génératrices et les matrices de contrôle d’un code de Hamming (n, k).
3. Ajouter le calcul du syndrome au programme et corriger les éventuelles erreurs.

### 1. Algorithme de Construction de la Matrice Génératrice G et de Contrôle H d'un Code de Hamming
Entrée :

- m : Nombre de bits de parité.
- n : Longueur totale du mot de code.
- k: Nombre de bits d'information. Longueur du massage

Sorties :

- G : Matrice génératrice (taille k×n).
- H : Matrice de contrôle (taille m×n).

Étapes :

1. __Créer les matrices identité et de parité__
    - On crée la matrice identité et la matrice de parité en trouvant les puissances de 2 pour les bits de parité et les autres
        positions sont pour les bits d'information

2. __Construction de la matrice P__ 
    - Chaque colonne de P correspond à un bit de parité (p1,p2,…).
    - Chaque ligne de P correspond à un bit d'information (i1,i2,…).
    - La relation est déterminée par un ET binaire (&) :
        - Si un bit d'information est couvert par un bit de parité, la valeur dans P est 1, sinon 0.

3. __Construire la matrice génératrice G :__

    - Créer une matrice identité I_k​ de taille k×k.
    - Transposer P pour obtenir P^T, de taille k×m.
    - Concaténer I_k​ et P^T horizontalement pour former G=[I_k∣P^T]

4. __Vérification de la validité de G et H :__

    - Afin de s'assurer que nos matrices sont correctes, on peut calculer G*H^T et la matrice qui en résulte doit valoir 0 pour chaque ij de la matrice 
        

In [ ]:
#!pip install matplotlib

In [2]:
from itertools import product
import math
import matplotlib.pyplot as plt
import numpy as np

### 2. Création des matrices génératrices et des matrices de contrôle

In [64]:
def generator_matrix(n,k):
    '''
    Cette fonction génère la matrice génératrice G d'un code linéaire de dimension k et de longueur n.
    '''
    # On commence a faire la matrice identité I_k
    id_matrix = np.eye(k, dtype=int)

    # On crée la matrice de positions des bits d'information et de parité
    positions = np.arange(1, n + 1)
    # La position des bits de parité sont les puissances de 2 donc il faut prendre le log2 des positions
    parity_positions = positions[np.log2(positions) % 1 == 0] 
    # Les positions des bits d'information sont les positions qui ne sont pas des puissances de 2
    info_positions = [pos for pos in positions if pos not in parity_positions]
    

    parity_matrix = []
    for parity in parity_positions:
        # On crée une ligne de la matrice de parité en fonction de la position du bit de parité et des bits d'information
        parity_row = [(parity & info) != 0 for info in info_positions]
        parity_matrix.append(parity_row)

    # Concaténation de I_k et P^T pour former G (.T = Transposer pour P^T)
    parity_matrix = np.array(parity_matrix).T 
    # hstack permet de concaténer horizontalement les deux matrices    
    generator_matrix = np.hstack((id_matrix, parity_matrix))
    return generator_matrix

def control_matrix(matrix,n,k):
    '''
    Fonctions qui génère la matrice de contrôle
    '''
    parity_part = []

    # Parcourt les indices des colonnes de parité
    for col_idx in range(k, n): 
        parity_column = matrix[:, col_idx]  # Extrait la colonne correspondante
        parity_part.append(parity_column)

    parity_part = np.array(parity_part).T 
    control_matrix = np.hstack((parity_part.T, np.eye(n - k, dtype=int)))

    return control_matrix

def validate_matrices(G, H):
    """
    Vérifie si G⋅H^T = 0, garantissant l'orthogonalité entre G et H.
    """
    result = np.dot(G, H.T) % 2  # Multiplication matricielle modulo 2
    if np.array_equal(result, np.zeros_like(result)):
        print("G et H sont valides : G⋅H^T = 0")
    else:
        print("G et H ne sont pas valides : G⋅H^T ≠ 0")
    return result



In [65]:
N_K = [(7,4), (15,11), (31,26)]

for (n, k) in N_K:
    gen_matrix = generator_matrix(n, k)
    c_matrixes = control_matrix(gen_matrix, n, k)
    validation = validate_matrices(gen_matrix, c_matrixes)
    print(f"n = {n}, k = {k}")
    print("Generator matrix :")
    print(gen_matrix)
    print("Control matrix :")
    print(c_matrixes)
    print("Validation :")
    print(validation)



G et H sont valides : G⋅H^T = 0
n = 7, k = 4
Generator matrix :
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 0 1 1]
 [0 0 0 1 1 1 1]]
Control matrix :
[[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]
Validation :
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
G et H sont valides : G⋅H^T = 0
n = 15, k = 11
Generator matrix :
[[1 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]
Control matrix :
[[1 1 0 1 1 0 1 0 1 0 1 1 0 0 0]
 [1 0 1 1 0 1 1 0 0 1 1 0 1 0 0]
 [0 1 1 1 0 0 0 1 1 1 1 0 0 1 0]
 [0 0 0 0 1 1 1 1 1 1 1 0 0 0 1]]
Validation :
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
G et H sont valides : G⋅H^T = 0
n

### 3. Calcul du syndrome et correction des erreurs

In [63]:
def calculate_syndrome(received_word, control_matrix):
    """
    Calcule le syndrome d'un mot reçu.
    """
    # Multiplication matrice et vecteur avec modulo 2
    syndrome = np.dot(control_matrix, received_word) % 2
    return syndrome


def correct_error(received_word, control_matrix):
    """
    Corrige une erreur unique dans un mot reçu basé sur le syndrome.
    """
    # Calculer le syndrome
    syndrome = calculate_syndrome(received_word, control_matrix)
    
    # Si le syndrome est nul, pas d'erreur
    if not np.any(syndrome):
        return received_word, syndrome, -1  # -1 indique pas d'erreur

    # Chercher la colonne de H correspondant au syndrome
    for idx, column in enumerate(control_matrix.T):
        if np.array_equal(column, syndrome):
            # Corriger l'erreur en inversant le bit à l'indice `idx`
            received_word[idx] ^= 1  # XOR pour inverser le bit
            return received_word, syndrome, idx  # Retourne le mot corrigé, le syndrome et l'index corrigé
    
    # Si aucune correspondance n'est trouvé
    return received_word, syndrome, None  # None indique une erreur non corrigible



gen_matrix = generator_matrix(7, 4)
c_matrixes = control_matrix(gen_matrix, 7, 4)

sequence = [1, 0, 1, 1, 0, 1, 0]
wrong_sequence = sequence.copy()
wrong_sequence[2] = wrong_sequence[2] ^ 1 

corrected_sequence, syndrome, error_index = correct_error(wrong_sequence, c_matrixes)

print(f"Original sequence : {sequence}")
print(f"Wrong sequence : {wrong_sequence}")
print(f"Syndrome : {syndrome}")
print(f"Corrected sequence : {corrected_sequence}")
print(f"Error index : {error_index}")
print(f"syndrom correct : {calculate_syndrome(corrected_sequence, c_matrixes)}")



Original sequence : [1, 0, 1, 1, 0, 1, 0]
Wrong sequence : [1, 0, 1, 1, 0, 1, 0]
Syndrome : [0 1 1]
Corrected sequence : [1, 0, 1, 1, 0, 1, 0]
Error index : 2
syndrom correct : [0 0 0]
